# Kode for datafortelling om operative mål i PO Pensjon
Målingene legges manuelt inn i et excel dokument som må lastes opp som "../data/Måleparametere PO Pensjon 2022.csv" ved oppdatering. NB! For å lagre excel-ark som csv (UTF-8) må man åpne det i skrivebordsappen (ikke i nettleser) først. 

In [ ]:
import sys
import importlib
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go
from typing import Dict

import datastory as ds

sys.path.append('../lib')
import operative as opera
importlib.reload(opera)

In [ ]:
df = pd.read_csv("../data/Variabler til målinger.csv", sep=";", decimal=",", header=17, usecols=list(range(7)))

In [ ]:
df = df.dropna().reset_index(drop=True)
df.columns = ["aar", "margin", "std_err", "t", "p_t", "conf_95_min", "conf_95_max"]


In [ ]:
def clean_numbers(x):
    if isinstance(x, float):
        return x
    if "%" in str(x):
        tmp = x[:-2]
        x = str(float(tmp)/100)
    if "-" in str(x):
        x = "0"
    return "".join(x.split(" "))

# Oversett prosent til desimaltall
df.loc[24:28, "margin"] = df.loc[24:28, "margin"].apply(clean_numbers)

# string to float
number_cols = ["margin", "std_err", "t", "p_t", "conf_95_min", "conf_95_max"]
for col in number_cols:
    df.loc[:, col] = df.loc[:, col].apply(lambda x: x.replace(',', '.')).astype("float")

In [ ]:
# df = (df
#       .drop(list(range(0,2))
#           + list(range(6,14))
#           + list(range(18,24)) 
#           + list(range(28,30)) 
#           + list(range(34,38)) 
#           + list(range(42,46))
#           + list(range(50,56))
#           + list(range(58,60))
#           + [62])
#       .reset_index(drop=True)
#      )

In [ ]:
df["maal"] = (  ["Søkt på sist søknad uføretrygd"] * 4 
             + ["Søkt på sist søknad alderspensjon"] * 4
             + ["Uføre, uten ventetid"] * 4
             + ["Alder, uten ventetid"] * 4
             + ["Tilfredshet uføretrygd"] * 4
             + ["Tilfredshet alderspensjon"] * 4
             + ["Selvbetjeningsgrad uføre"] * 2
             + ["Selvbetjeningsgrad alder"] * 2
            )

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)

In [ ]:
df.dtypes

# Make dataproduct

In [ ]:
df.head()

In [ ]:
from google.cloud.bigquery import Client, LoadJobConfig

table_id = f'teampensjon-prod-cebd.operative_mal.tilfredshet'
job_config = LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client = Client(project="teampensjon-prod-cebd")

job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
job.result()